In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib

## Data import 및 확인

In [ ]:
df = pd.read_csv("five_minute/한국전력거래소_5분단위 전력수급현황_20230430.csv", encoding='cp949', index_col='기준일시', parse_dates=True)
df
# 종속변수는 현재수요(MW)로 설정한다.

In [ ]:
df[['현재수요(MW)']].plot(figsize =(20,10));

In [ ]:
from statsmodels.tsa.statespace.tools import diff
diff(df["현재수요(MW)"], k_diff=1).plot(figsize=(12,8));

# Target data 추출

## train 데이터 추출

In [ ]:
# train기간
train = df.loc[:'2023-03-12 23:55:00']

In [ ]:
train.to_csv("train.csv")

In [ ]:
train

In [ ]:
X_train = train.drop("현재수요(MW)", axis=1)
X_train.to_csv("X_train.csv")
y_train = train['현재수요(MW)']
y_train.to_csv("y_train.csv")


In [ ]:
X_train.shape, y_train.shape

## test 데이터 추출

In [ ]:
# test기간
test = df.loc['2023-03-13 00:00:00':'2023-03-19 23:55:00']
test.to_csv("test.csv")

In [ ]:
test

In [ ]:
X_test = test.drop("현재수요(MW)", axis=1)
X_test.to_csv("X_test.csv")

In [ ]:
y = test['현재수요(MW)']
y.to_csv("y.csv")


In [ ]:
X_test.shape, y.shape

## train, test 분포 비교

In [ ]:
train.plot(figsize = (12,8));

In [ ]:
test.plot(figsize = (12,8));

# Ensemble

## RandomForest(GridSearch)

In [ ]:
# 기상데이터 import
df_final = pd.read_csv("meteorological/df_final.csv", index_col=0, parse_dates=True) # 기상데이터 total 최종본
df_final.index.freq = '5T'
train = pd.read_csv("meteorological/train.csv", index_col=0, parse_dates=True) # 2022-04-01 00:00:00 ~ 2023-03-12 23:55:00 까지의 데이터
train.index.freq = '5T'
X_train = pd.read_csv("meteorological/X_train.csv", index_col=0, parse_dates=True)
X_train.index.freq='5T'
y_train = pd.read_csv("meteorological/y_train.csv", index_col=0, parse_dates=True)
y_train.index.freq='5T'
test = pd.read_csv("meteorological/test.csv", index_col=0, parse_dates=True) # 2023-03-13 00:00:00':'2023-03-19 23:55:00' 까지의 데이터
test.index.freq = '5T'
X_test = pd.read_csv("meteorological/X_test.csv", index_col=0, parse_dates=True)
X_test.index.freq = '5T'
y = pd.read_csv("meteorological/y.csv", index_col=0, parse_dates=True)
y.index.freq = '5T'

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import train_test_split

In [ ]:
def get_score_splited_train(model, xtrain, xtest, ytrain, ytest):
    A = model.score(xtrain, ytrain)
    B = model.score(xtest,ytest)
    pred = model.predict(xtest)
    C = mape(ytest, pred)

    print(f"ACC train : {A:.4f}, test : {B:.4f}, mape : {C:.4f}" )

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X_train, y_train)

In [ ]:

from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트 회귀 모델을 준비합니다.
rf = RandomForestRegressor(random_state=42)

# 탐색할 하이퍼파라미터의 범위를 지정합니다.
param_grid = {
    'n_estimators': [50, 100, 200],    # 트리의 개수
    'max_depth': [None, 10, 20, 30],   # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],   # 노드를 분할하기 위한 최소한의 샘플 데이터 수
    'min_samples_leaf': [1, 2, 4],     # 리프 노드가 되기 위한 최소한의 샘플 데이터 수
    'bootstrap': [True, False]         # 부트스트랩(중복 허용 샘플링) 사용 여부
}

# 그리드 탐색을 준비합니다.
# cv는 cross-validation의 fold 수를 나타냅니다.
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)

# 데이터 X와 타겟 y에 대해 그리드 탐색을 수행합니다.
grid_search.fit(xtrain, ytrain)

# 최적의 하이퍼파라미터를 출력합니다.
print(grid_search.best_params_)


In [ ]:
best_rf = RandomForestRegressor(random_state=42, bootstrap=True, max_depth=None, min_samples_leaf= 1, min_samples_split=2, n_estimators=200)
best_rf.fit(xtrain,ytrain)
get_score_splited_train(best_rf, xtrain, xtest, ytrain, ytest)
prediction = best_rf.predict(X_test)
mape(y, prediction)
feature_importances = pd.DataFrame(best_rf.feature_importances_,
                                   index = xtrain.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)


In [ ]:
prediction = pd.Series(prediction)
prediction.index = y.index
prediction

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(prediction, label = 'rf')
plt.plot(y, label = 'y')
plt.xlabel(y.index)
plt.legend()
plt.show()

In [ ]:
# 모든 데이터 학습
best_rf.fit(X_train,y_train)

In [ ]:
prediction = best_rf.predict(X_test)

In [ ]:
print(mape(y, prediction))

In [ ]:
feature_importances = pd.DataFrame(best_rf.feature_importances_,
                                   index = xtrain.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)


In [ ]:
prediction = pd.Series(prediction)
prediction.index = y.index
prediction

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(prediction, label = 'rf')
plt.plot(y, label = 'y')
# plt.xlabel(y.index)
plt.legend()
plt.show()

## Time Series 변수 생성

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from sklearn.preprocessing import PowerTransformer
# pt = PowerTransformer(method='yeo-johnson')
# df['column'] = pt.fit_transform(df[['column']])

scaler= MinMaxScaler()


In [ ]:
# Time Series 분석에 따른 target time 변수
target_time = [25,68,156,288,576]

In [ ]:
target_list = list(X_train.columns)
target_list

In [ ]:
TES_X_train = X_train.copy()
TES_X_test = X_test.copy()

In [ ]:
# 0 값의 오류로 인한 로그변환
for col in target_list:
    TES_X_train[col] = scaler.fit_transform(TES_X_train[[col]])
    TES_X_test[col] = scaler.fit_transform(TES_X_test[[col]])

In [ ]:
TES_X_train.head()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tqdm import tqdm
def make_TES_se_add(target_time, col, TES_X_train, TES_X_test):
    for i in tqdm(target_time):
        model_add = ExponentialSmoothing(TES_X_train[col], trend='add', seasonal='add', seasonal_periods=i).fit()
        TES_X_train[f'TESadd_add{i}{col}'] = model_add.fittedvalues
        model_add = ExponentialSmoothing(TES_X_test[col], trend='add', seasonal='add', seasonal_periods=i).fit()
        TES_X_test[f'TESadd_add{i}{col}'] = model_add.fittedvalues

#
#     for i in tqdm(target_time):
#         model_add = ExponentialSmoothing(TES_X_train[col], trend='mul', seasonal='add', seasonal_periods=i).fit()
#         TES_X_train[f'TESmul_add{i}{col}'] = model_add.fittedvalues
#         model_add = ExponentialSmoothing(TES_X_test[col], trend='mul', seasonal='add', seasonal_periods=i).fit()
#         TES_X_test[f'TESmul_add{i}{col}'] = model_add.fittedvalues
    return TES_X_train, TES_X_test
#
# # TES_X_train.tail()

In [ ]:
def make_TES_mul(target_time, col, TES_X_train, TES_X_test):
    for i in tqdm(target_time):
        model_add = ExponentialSmoothing(TES_X_train[col], trend='mul', seasonal='add', seasonal_periods=i).fit()
        TES_X_train[f'TESmul_add{i}{col}'] = model_add.fittedvalues
        model_add = ExponentialSmoothing(TES_X_test[col], trend='mul', seasonal='add', seasonal_periods=i).fit()
        TES_X_test[f'TESmul_add{i}{col}'] = model_add.fittedvalues


    for i in tqdm(target_time):
        model_mul = ExponentialSmoothing(TES_X_train[col], trend='mul', seasonal='mul', seasonal_periods=i).fit()
        TES_X_train[f'TESmul_mul{i}{col}'] = model_mul.fittedvalues
        model_mul = ExponentialSmoothing(TES_X_test[col], trend='mul', seasonal='mul', seasonal_periods=i).fit()
        TES_X_test[f'TESmul_mul{i}{col}'] = model_mul.fittedvalues
    return TES_X_train, TES_X_test

In [ ]:
for col in target_list:
    TES_X_train, TES_X_test = make_TES_se_add(target_time, col, TES_X_train, TES_X_test)

In [ ]:
TES_X_train.tail()

In [ ]:
TES_X_test.tail()

In [ ]:
# 모든 데이터 학습
import pickle
def run_model(TES_X_train, TES_X_test, y_train, y):
    best_rf = RandomForestRegressor(random_state=42, bootstrap=True, max_depth=None, min_samples_leaf= 1, min_samples_split=2, n_estimators=200)
    best_rf.fit(TES_X_train,y_train)
    prediction = best_rf.predict(TES_X_test)
    print(f"mape : {mape(y, prediction):.4f}")
    feature_importances = pd.DataFrame(best_rf.feature_importances_,
                                       index = TES_X_test.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)

    prediction = pd.Series(prediction)
    prediction.index = y.index
    prediction
    plt.figure(figsize=(12,8))
    plt.plot(prediction, label = 'rf')
    plt.plot(y, label = 'y')
    plt.xlabel(y.index)
    plt.legend()
    plt.show()
    with open('meteorological/model/rf/all_feature_rf_model.pkl', 'wb') as file:
        pickle.dump(best_rf, file)
    return

In [ ]:
run_model(TES_X_train, TES_X_test,y_train, y)

In [ ]:
feature_importances

## 단계선택법

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
# 단계선택법



def rfecv_run_model(TES_X_train, TES_X_test, y_train, y):
    best_rf = RandomForestRegressor(random_state=42, bootstrap=True, max_depth=None, min_samples_leaf=1,
                                    min_samples_split=2, n_estimators=200)
    rfecv = RFECV(estimator=best_rf)

    rfecv.fit(TES_X_train, np.ravel(y_train))
    prediction = rfecv.predict(TES_X_test)
    print(f"mape : {mape(y, prediction):.4f}")
    # feature_importances = pd.DataFrame(rfecv.feature_,
    #                                    index=TES_X_test.columns,
    #                                    columns=['importance']).sort_values('importance', ascending=False)
    # print(feature_importances)

    # prediction = pd.Series(prediction)
    # prediction.index = y.index
    # prediction
    # plt.figure(figsize=(12, 8))
    # plt.plot(prediction, label='rf')
    # plt.plot(y, label='y')
    # plt.xlabel(y.index)
    # plt.legend()
    # plt.show()
    with open('meteorological/model/rf/selected_features_rfecv_model.pkl', 'wb') as file:
        pickle.dump(rfecv, file)
    return rfecv

In [ ]:
help(RFECV)

In [ ]:
rfecv = rfecv_run_model(TES_X_train, TES_X_test,y_train, y)

In [ ]:
selected_features = TES_X_test.columns[rfecv.support_]

In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (accuracy)")
plt.plot(
    range(rfecv.min_features_to_select, len(rfecv.grid_scores_) + rfecv.min_features_to_select),
    rfecv.grid_scores_,
)

In [195]:
selected_features

Index(['이슬점온도', '기온', '상대습도', '강수량', '일조량', '지면온도', '풍속', '기압', '시정(가시거리)',
       'TESadd_add25이슬점온도', 'TESadd_add68이슬점온도', 'TESadd_add156이슬점온도',
       'TESadd_add288이슬점온도', 'TESadd_add576이슬점온도', 'TESadd_add25기온',
       'TESadd_add68기온', 'TESadd_add156기온', 'TESadd_add288기온',
       'TESadd_add576기온', 'TESadd_add25상대습도', 'TESadd_add68상대습도',
       'TESadd_add156상대습도', 'TESadd_add288상대습도', 'TESadd_add576상대습도',
       'TESadd_add25강수량', 'TESadd_add68강수량', 'TESadd_add156강수량',
       'TESadd_add576강수량', 'TESadd_add25일조량', 'TESadd_add68일조량',
       'TESadd_add156일조량', 'TESadd_add288일조량', 'TESadd_add576일조량',
       'TESadd_add25지면온도', 'TESadd_add68지면온도', 'TESadd_add156지면온도',
       'TESadd_add288지면온도', 'TESadd_add576지면온도', 'TESadd_add25풍속',
       'TESadd_add68풍속', 'TESadd_add156풍속', 'TESadd_add288풍속',
       'TESadd_add576풍속', 'TESadd_add25기압', 'TESadd_add68기압',
       'TESadd_add156기압', 'TESadd_add288기압', 'TESadd_add576기압',
       'TESadd_add25시정(가시거리)', 'TESadd_add68시정(가시거리)', 'TESadd_a